# Data Processing with Pandas - Spotify Top 50 Tracks of 2020

Ever wondered what makes a song a global hit? This analysis dives into Spotify's Top 50 tracks of 2020 to uncover the hidden musical DNA that defines popular music, from energy levels to danceability, and how these attributes shape our listening experiences.

### Key Questions to Answer:

1. What are the dominant genres in 2020's top charts?
2. Are there specific audio features that consistently appear in popular tracks?
3. Can we identify distinct 'types' of hit songs based on their characteristics?

In [ ]:
# Import neccessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%load_ext blackcellmagic

## 1. Load & Display Data

First, we load the data and inspect its structure to ensure it's ready for analysis. We'll check the first few rows and column data types.

The CSV file has its own index column, so we instruct pandas to use the first column (index 0)

In [ ]:
# Load CSV file and set the first column as the DataFrame index
df = pd.read_csv("spotify/spotifytoptracks.csv", index_col=0)
df.head()

In [ ]:
# Display basic info about the generated data
print(f"\nDataFrame Info:")
df.info()

#### Initial Data Overview:

- Dataset contains 50 rows and 16 columns
- There are no missing values
- Provides an overview of popular music attributes: energy, danceability, key, loudness, acousticness, speechiness, instrumentalness, liveness, valence, tempo, duration_ms.
- **Note:** duration_ms is in miliseconds. We will convert it to minutes and seconds later
- Provides genres: R&B/Soul, Alternative/Indie, Hip-Hop/Rap, Dance/Electronic, etc.
- The dataset is relatively small, consuming only 6.6+ KB of memory, making it efficient to work with.

## 2. Data Cleaning

Standardize text formatting, remove redundant columns, and address missing and duplicate values.

### 2.1. String Formatting

In [ ]:
# Capitalize the first letter of each word in selected columns
text_cols = ["artist", "album", "track_name"]
for col in text_cols:
    df[col] = df[col].str.title()

### 2.2. Removing Columns

Removing track_id that is a unique identifier not needed for aggregate trend analysis.

In [ ]:
# Eliminating "track_id" since it's irrelevant for analysis
df.drop("track_id", axis=1, inplace=True)

In [ ]:
df.head()

### 2.3. Handling missing values

As previously noted, this dataset contains no missing values. This section is included for future re-use, when dataset with missing values require processing.

Display existing missing value rows before deletion.

In [ ]:
def show_missing_rows(df):
    """
    Check and display rows with missing values in the DataFrame
    """
    missing_rows = df[df.isnull().any(axis=1)]
    if not missing_rows.empty:
        print("Rows with missing values found:")
        display(missing_rows)
    else:
        print("No missing values found.")

In [ ]:
show_missing_rows(df)

In [ ]:
# Remove any rows from the DataFrame that contain missing values
# df.dropna(inplace=True)

### 2.4. Display and remove duplicate samples and features

Display existing duplicate values before deletion.

In [ ]:
def handle_duplicate_df(df):
    """
    Detects and removes duplicate rows and columns from the DataFrame.
    """
    duplicate_rows = df[df.duplicated(keep=False)]
    duplicate_cols = df.columns[df.T.duplicated()]

    if not duplicate_rows.empty:
        print("Duplicate samples found:")
        display(duplicate_rows)
    else:
        print("No duplicate samples found.")

    if len(duplicate_cols) > 0:
        print("Duplicate features found:", list(duplicate_cols))
    else:
        print("No duplicate features found.")

    df = df.drop_duplicates().loc[:, ~df.T.duplicated()]
    return df

In [ ]:
df = handle_duplicate_df(df)

### 2.5. Treating outliers

#### 2.5.1. Perform normalization

Performed min-max normalization on all numeric audio features to scale them into a [0, 1] range. 
This transformation ensures proportional feature contribution by minimizing the influence of high-magnitude variables (like duration_ms).

In [ ]:
# Import scaler to transform numeric data to a range between 0 and 1.
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Select only numeric columns
df_numeric = df.select_dtypes(include=[np.number])

# Normalize using MinMaxScaler
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_numeric), columns=df_numeric.columns)

In [ ]:
df_scaled.head()

#### 2.5.2. Count upper and lower bound outliers

In [ ]:
def count_all_outliers(df):
    outlier_dict = {}
    for col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        lower_outliers = (df[col] < lower_bound).sum()
        upper_outliers = (df[col] > upper_bound).sum()

        outlier_dict[col] = {
            "Upper Outliers": upper_outliers,
            "Lower Outliers": lower_outliers  
        }

    outlier_df = pd.DataFrame.from_dict(outlier_dict, orient="index").T
    return outlier_df

In [ ]:
outlier_summary_df = count_all_outliers(df_scaled)
outlier_summary_df

#### 2.5.2. Plot the outliers

In [ ]:
# Plot the boxplot
df_scaled.boxplot(figsize=(10, 6), rot=45)
plt.title("Boxplot of Normalized Feature Columns")
plt.tight_layout()
plt.show()

#### Insights 
- **Outlier Presence:** Features like acousticness and speechiness show a high concentration of outliers on the upper end, suggesting some tracks have unusually high values for these characteristics.
- Energy, key, valence and tempo appear to be more uniformly distributed, suggesting lower variability and fewer anomalies.
- It's also notable that Instrumentalness has a median near zero, indicating that **most tracks are not highly instrumental**.

**We'll keep these outliers for this analysis, as top music tracks often have extreme values.**

## 3. Exploratory Data Analysis

### 3.1. Summary of Dataset

In [ ]:
def summarize_dataframe(df):
    """
    Generate and display a summary of the dataset including
    number of observations, number of features, and lists
    of categorical and numeric columns.
    """
    categorical = df.select_dtypes(include="object").columns.tolist()
    numerical = df.select_dtypes(include="number").columns.tolist()

    summary = pd.DataFrame(
        {
            "Description": [
                "Number of observations",
                "Number of features",
                f"Categorical features ({len(categorical)})",
                f"Numeric features ({len(numerical)})",
            ],
            "Value": [
                df.shape[0],
                df.shape[1],
                ", ".join(categorical),
                ", ".join(numerical),
            ],
        }
    )

    pd.set_option("display.max_colwidth", None)
    display(summary)

In [ ]:
summarize_dataframe(df)

### 3.2. TOP Artists Overview

In [ ]:
# Count how many times each artist appears
artist_counts = df["artist"].value_counts()

In [ ]:
def summarize_artists_table(artist_counts, min_tracks=2):
    """
    Return a DataFrame of artists with > 1 tracks.
    Adds a 'status' column to indicate top artist(s).
    """
    max_tracks = artist_counts.max()

    # Filter artists with ≥ min_tracks
    popular_artists = artist_counts[artist_counts >= min_tracks]
    df_popular = popular_artists.reset_index()
    df_popular.columns = ["artist", "num_top_tracks"]

    # Tag top artist(s)
    df_popular["status"] = df_popular["num_top_tracks"].apply(
        lambda x: "Top Artist" if x == max_tracks else "Multiple Tracks"
    )

    print(
        f"Total Artists: {len(df_popular) + (artist_counts == 1).sum()}\n"
        f"Top artist(s) with {max_tracks} track(s): "
        f"{', '.join(df_popular[df_popular["num_top_tracks"] == max_tracks]["artist"])}"
    )
    print(
        f"Artists with ≥ {min_tracks} top track(s): {len(df_popular)}\n"
        f"Artists with 1 top track: {(artist_counts == 1).sum()}\n"
    )

    return df_popular.sort_values(by="num_top_tracks", ascending=False)

In [ ]:
df_summary = summarize_artists_table(artist_counts)
display(df_summary)

#### Insights 
- There are 40 artists in the Top 50 list.
- The most popular artists: **Billie Eilish, Dua Lipa, Travis Scott** each having 3 of their tracks in the list.
- Justin Bieber, Harry Styles, Lewis Capaldi, Post Malone has 2 of their tracks on the list.

### 3.3. Number of Artists by Track Visualization

In [ ]:
# How many artists fall into each count bucket
artist_freq = artist_counts.value_counts().sort_index()

# Create a DataFrame for plotting
df_artist_freq = artist_freq.reset_index(name="num_artists")
df_artist_freq.columns = ["track_count", "num_artists"]

In [ ]:
# Plotting
plt.figure(figsize=(6, 4))
ax = sns.barplot(
    x="track_count",
    y="num_artists",
    data=df_artist_freq,
    palette="pastel",
    hue="track_count",
    legend=False,
)

for container in ax.containers:
    ax.bar_label(container, fmt="%d", label_type="center", padding=-2)

plt.title("Number of Artists by Track Count")
plt.xlabel("Number of Tracks")
plt.ylabel("Number of Artists")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

#### Insights 
- The vast majority of artists in Top 50 (33 out of 40) have only one track included.
- 7 artists have multiple tracks, with 4 artists contributing two tracks and 3 artists contributing three tracks.

### 3.4. TOP Albums

In [ ]:
def summarize_top_albums(df, min_tracks=2):
    """
    Summarize albums that have > 1 track in top tracks.
    Returns a sorted DataFrame with album name, artist, and number of top tracks.
    """
    # Count number of top tracks per album
    album_counts = df["album"].value_counts()
    multi_track_albums = album_counts[album_counts >= min_tracks].index

    # Filter only albums meeting the threshold
    df_multi = df[df["album"].isin(multi_track_albums)]

    # Group summary by album
    df_album_summary = (
        df_multi.groupby("album")
        .agg(artist=("artist", "first"), num_top_tracks=("track_name", "count"))
        .reset_index()
        .sort_values(by="num_top_tracks", ascending=False).reset_index(drop=True)
    )

    # Reorder columns
    df_album_summary = df_album_summary[["album", "artist", "num_top_tracks"]]

    return df_album_summary

In [ ]:
display(summarize_top_albums(df))

In [ ]:
unique_albums = df["album"].nunique()
print(f"Total number of unique albums: {unique_albums}")

#### Insights 
- There are 45 albums appearing in the Top 50 list.
- 4 Albums appear more than one time
- **'Future Nostalgia'** by Dua Lipa is the most popular one, contributing 3 times.

### 3.5. Feature Overview

#### 3.5.1. Danceability

**High Danceability**

Danceability values **above 0.7** are used as a threshold to define "high." This implies that the track possesses a strong, consistent rhythm, stable tempo, and overall feel that makes it easy to dance to.

In [ ]:
def high_danceability_tracks(df, threshold=0.7):
    """
    Prints and returns tracks with danceability above the given threshold.
    """
    tracks = df[df["danceability"] > threshold].sort_values(
        "danceability", ascending=False
    )
    print(
        f"There are {len(tracks)} track(s) that have a danceability score above {threshold}\n"
    )
    return tracks.head()

In [ ]:
high_danceability_tracks(df)

#### Insights
- A significant majority **64%** (32 out of 50) of 2020's top tracks contribute high danceability scores (above 0.7)
- The most danceable track: Cardi B's Hip-Hop/Trap hit **'WAP' (Feat. Megan Thee Stallion)**, with score of 0.935

**Low Danceability**

Danceability values **below 0.40** indicate that a track has a less prominent beat, an irregular tempo, or a structure that doesn't lend itself easily to dancing.

In [ ]:
def low_danceability_tracks(df, threshold=0.4):
    """
    Prints and returns tracks with danceability below the given threshold.
    """
    tracks = df[df["danceability"] < threshold].sort_values("danceability")
    print(
        f"There are {len(tracks)} track(s) that have a danceability score below {threshold}\n"
    )
    return tracks.head()

In [ ]:
low_danceability_tracks(df)

#### Insights
- Only **2%** (1 out of 50) registered with a low danceability score, indicating its rarity among chart-topping hits.
- Lowest Danceability track was Chamber pop hit **'Lovely (With Khalid)'** by Billie Eilish with score of 0.351

#### 3.5.2. Loudness

**High Loudness**

0 dB represents the maximum possible loudness, values approaching it: **-1.0 dB to -5.0 dB** are considered very high.

In [ ]:
def high_loudness_tracks(df, threshold=-5):
    """
    Prints and returns tracks with loudness above the given threshold.
    """
    tracks = df[df["loudness"] > threshold].sort_values("loudness", ascending=False)
    print(
        f"There are {len(tracks)} track(s) that have a loudness score above {threshold}\n"
    )
    return tracks.head()

In [ ]:
high_loudness_tracks(df)

#### Insights
- Nearly **40%** of the Top 50 tracks (19 songs) register high loudness scores
- Pop hit **'Tusa'** by Karol G stands out as the loudest track in the dataset. (-3.280 dB)

**Low Loudness**

Loudness score is generally considered low when it is: -15.0 dB to -25.0 dB
These values indicate a track that is significantly quieter.

Since there are no values in our dataset in this range, we calculate "low" relative to the other tracks in Top 50.

In [ ]:
print("\n--- Quartiles for Loudness ---")
loudness_q1 = df["loudness"].quantile(0.25)
loudness_median = df["loudness"].quantile(0.50)
loudness_q3 = df["loudness"].quantile(0.75)

print(f"25th Percentile (Q1) Loudness: {loudness_q1:.3f} dB")
print(f"50th Percentile (Median) Loudness: {loudness_median:.3f} dB")
print(f"75th Percentile (Q3) Loudness: {loudness_q3:.3f} dB")

- Will use the rounded 25th Percentile of **-8 dB**

In [ ]:
def low_loudness_tracks(df, threshold=-8):
    """
    Prints and returns tracks with loudness below the given threshold.
    """
    tracks = df[df["loudness"] < threshold].sort_values("loudness")
    print(
        f"There are {len(tracks)} track(s) that have a loudness score below {threshold}\n"
    )
    return tracks.head()

In [ ]:
low_loudness_tracks(df)

#### Insights
- Top 50 generally comprises loud tracks
- **18%** (9 out of 50) falling into the 'very low' loudness category relative to the rest of our dataset.
- The most silent track is the Pop hit **'Everything I Wanted'** by Billie Eilish. Its low loudness score -14.5 dB

#### 3.5.3. Length

Calculate the longest and shortest tracks in the list

In [ ]:
def format_duration(duration_ms):
    """
    Convert duration from milliseconds to mm:ss format.
    """
    seconds = duration_ms / 1000
    minutes = int(seconds // 60)
    seconds = int(seconds % 60)
    return f"{minutes}:{seconds:02d}"

In [ ]:
def get_extreme_duration_tracks(df):
    """
    Returns the longest and shortest tracks in mm:ss format.
    """
    longest = df.loc[df["duration_ms"].idxmax()]
    shortest = df.loc[df["duration_ms"].idxmin()]

    results = {
        "Type": ["Longest", "Shortest"],
        "Track Name": [longest["track_name"], shortest["track_name"]],
        "Artist": [longest["artist"], shortest["artist"]],
        "Duration (mm:ss)": [
            format_duration(longest["duration_ms"]),
            format_duration(shortest["duration_ms"]),
        ],
    }

    return pd.DataFrame(results)

In [ ]:
get_extreme_duration_tracks(df)

#### 3.5.4. Genre

Analyze genre distribution

In [ ]:
def analyze_genres(df):
    """
    Analyze genre data to identify:
    - The most frequently contributed genre and its track count
    - Genres that appear only once, along with their count
    - Total number of distinct genres in the dataset
    """
    genre_counts = df["genre"].value_counts()
    most_popular_genre = genre_counts.idxmax()
    most_popular_count = genre_counts.max()
    one_song_genres = genre_counts[genre_counts == 1]
    unique_genres = df["genre"].nunique()

    print(f"Most popular genre: '{most_popular_genre}' with {most_popular_count} tracks.")
    print(f"Genres with only one song ({one_song_genres.count()} total):\n{one_song_genres}\n")
    print(f"Total number of unique genres: {unique_genres}\n")

In [ ]:
# Visualize genre distribution
def plot_genre_distribution(df):
    genre_counts = df["genre"].value_counts()
    genre_counts.plot(kind="barh", figsize=(8, 4), color="limegreen")
    plt.title("Genre Distribution")
    plt.xlabel("Number of Tracks")
    plt.ylabel("Genre")
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
analyze_genres(df)
plot_genre_distribution(df)

#### Insights
- Dominant genres: Pop **28%** (14 tracks) and Hip-Hop/Rap **20%** (10 tracks) together account for nearly half of the most streamed songs.
- Dataset includes 16 distinct genres, 10 of which are represented by only one song.

### 3.6. Correlation Overview

In [ ]:
def find_correlations(df, label, lower_bound, upper_bound):
    """
    Extract correlation values between numeric features in a specific range.

    Args:
        df (pd.DataFrame): Input DataFrame
        label: Label for the type of correlation
        lower_bound: Lower threshold
        upper_bound: Upper threshold

    Returns:
        pd.DataFrame: Filtered correlation matrix
    """
    numeric_df = df.select_dtypes(include="number")
    corr_matrix = numeric_df.corr()

    # Mask upper triangle to avoid duplicates
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    corr_matrix = corr_matrix.mask(mask)

    # Filter based on bounds
    filtered = corr_matrix[(corr_matrix > lower_bound) & (corr_matrix < upper_bound)]
    filtered = filtered.dropna(how="all").dropna(how="all", axis=1)

    # Replace NaNs with dash for display
    filtered = filtered.fillna("-")
    
    print(f"{label} features (correlation between {lower_bound} and {upper_bound}):") 


    return filtered

#### 3.6.1. Strongly Positive Corellation

In [ ]:
find_correlations(df, "Strongly positive", 0.7, 1.0)

#### 3.6.2. Strongly Negative Corellation

In [ ]:
find_correlations(df, "Strongly negative", -1.0, -0.6)

#### 3.6.3. Weak to no correlation

In [ ]:
find_correlations(df, "Weak to no correlation", -0.1, 0.1)

#### Insights
- **Energy ↔ Loudness:** With a correlation of 0.79, these two are closely linked—meaning louder tracks tend to be more energetic.
- **Energy ↔ Acousticness:** Correlation of -0.68 suggests that more acoustic tracks tend to be less energetic.


## 4. Statistical Comparison

This section examines the feature **(danceability, loudness, acousticness)** scores across top 4 music genres by:
- Comparing their mean feature values
- Visualizing score distributions
- Conducting pairwise t-tests to assess whether observed differences are statistically significant

The goal is to determine if these features varies meaningfully between: Pop, Hip-Hop/Rap, Dance/Electronic, and Alternative/Indie.

In [ ]:
# Define genre selection
selected_genres = ["Pop", "Hip-Hop/Rap", "Dance/Electronic", "Alternative/Indie"]

### 4.1. Danceability between genres

#### 4.1.1 Danceability Means Comparison

In [ ]:
# Average danceability scores for selected genres
genre_danceability = (
    df.groupby("genre")["danceability"]
    .mean()
    .loc[selected_genres]
    .sort_values(ascending=False)
)

print("Danceability scores comparison between selected genres:")
display(genre_danceability.to_frame())

#### 4.1.2. Danceability Score Distributio Plot

In [ ]:
# Violin plot of danceability distributions by genre
filtered_df = df[df["genre"].isin(selected_genres)]
genre_medians = filtered_df.groupby("genre")["danceability"].median()

plt.figure(figsize=(8, 5))
sns.violinplot(data=filtered_df, x="genre", y="danceability", inner="box")

# Annotate medians
for i, genre in enumerate(selected_genres):
    median = genre_medians[genre]
    plt.text(i, median + 0.1, f"Median: {median:.2f}", ha="center", color="red")

# Plot
plt.title("Danceability Score Distribution by Genre", fontsize=14)
plt.ylabel("Danceability")
plt.xlabel("Genre")
plt.xticks(rotation=15)
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

####  Insights
- Dance/Electronic leads with the highest median danceability at 0.79
- Dance/Electronic and Hip-Hop/Rap genres are less spread, meaning their tracks consistently aim for higher danceability.
- Alternative/Indie and Pop have wider distributions, displaying more variation across tracks — from danceable hits to mellow ballads.

#### 4.1.3. Danceability T-Test Comparison

General Hypotheses for Danceability Comparison Between Two Genres:

- Null Hypothesis (H₀): There is no difference in the mean danceability between the selected genre pairs.
- Alternative Hypothesis (H₁): There is a difference in the mean danceability between the two genres.

This is a two-tailed test, evaluating whether one genre tends to be more or less danceable than the other on average.

In [ ]:
# Pairwise T-tests for danceability between genres
from scipy.stats import ttest_ind

results = []

for i, genre_a in enumerate(selected_genres):
    for genre_b in selected_genres[i + 1:]:
        group_a = df[df["genre"] == genre_a]["danceability"]
        group_b = df[df["genre"] == genre_b]["danceability"]
        
        t_stat, p_value = ttest_ind(group_a, group_b, equal_var=False, nan_policy="omit")
        
        results.append({
            "Genre A": genre_a,
            "Genre B": genre_b,
            "T-Statistic": round(t_stat, 3),
            "P-Value": round(p_value, 4),
            "Significant": "Yes" if p_value < 0.05 else "No"
        })

ttest_df = pd.DataFrame(results)

print("Danceability Genre Pairwise T-Test Results:")
display(ttest_df)

#### Results:

Pop vs. Hip-Hop/Rap shows a statistically significant difference in danceability
- P-value: 0.0284 < 0.05 → We reject the null hypothesis
- **Hip-Hop/Rap is considered more danceable than Pop**

#### 4.1.4. Effect Size (Cohen’s d) for Two Independent Samples

We calculate effect size, which tells us how big/meaningful the difference is between these groups, beyond statistical significance.

In [ ]:
def cohens_d(group1, group2):
    """Compute Cohen's d for two independent samples."""
    mean_diff = np.mean(group1) - np.mean(group2)
    pooled_std = np.sqrt(
        ((np.std(group1, ddof=1) ** 2) + (np.std(group2, ddof=1) ** 2)) / 2
    )
    return round(mean_diff / pooled_std, 3)

In [ ]:
def interpret_effect_size(d):
    abs_d = abs(d)
    if abs_d < 0.2:
        return "Negligible"
    elif abs_d < 0.5:
        return "Small"
    elif abs_d < 0.8:
        return "Medium"
    else:
        return "Large"

In [ ]:
effect_sizes = []

for i, genre_a in enumerate(selected_genres):
    for genre_b in selected_genres[i + 1:]:
        group_a = df[df["genre"] == genre_a]["danceability"].dropna()
        group_b = df[df["genre"] == genre_b]["danceability"].dropna()
        
        d = cohens_d(group_a, group_b)
        interpretation = interpret_effect_size(d)

        effect_sizes.append({
            "Genre A": genre_a,
            "Genre B": genre_b,
            "Cohen's d": d,
            "Interpretation": interpretation

        })

effect_sizes_danceability_df = pd.DataFrame(effect_sizes)
display(effect_sizes_danceability_df)

A large effect size supports the conclusion that Hip-Hop/Rap is notably more danceable than Pop.


### 4.2. Loudness between genres

#### 4.2.1. Loudness Means Comparison

In [ ]:
# Average loudness scores for selected genres
genre_loudness = (
    df.groupby("genre")["loudness"]
    .mean()
    .loc[selected_genres]
    .sort_values()
)

print("Loudness scores comparison between selected genres:")
display(genre_loudness.to_frame())

#### 4.2.2. Loudnes Score Distribution

In [ ]:
# Violin plot of loudness distributions by genre
filtered_df = df[df["genre"].isin(selected_genres)]
genre_medians = filtered_df.groupby("genre")["loudness"].median()

plt.figure(figsize=(8, 5))
sns.violinplot(data=filtered_df, x="genre", y="loudness", inner="box")

# Annotate medians
for i, genre in enumerate(selected_genres):
    median = genre_medians[genre]
    plt.text(i, median + 0.1, f"Median: {median:.2f}", ha="center", color="red")

# Plot
plt.title("Loudness Score Distribution by Genre", fontsize=14)
plt.ylabel("Loudness")
plt.xlabel("Genre")
plt.xticks(rotation=15)
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

####  Insights
- Pop is the loudest genre overall by its median loudness of –5.27 dB
- Alternative/Indie presents a moderately dispersed distribution, generally lower loudness levels.
- Hip-Hop/Rap demonstrates the broadest loudness range.
- Dance/Electronic and Pop genres exhibit relatively tight loudness distributions, indicating consistency in audio across their respective tracks.

#### 4.2.3. Loudness T-Test Comparison

General Hypotheses for Loudness Comparison Between Two Genres:

- Null Hypothesis (H₀): There is no difference in the mean loudness between selected genre pairs.
- Alternative Hypothesis (H₁): There is a difference in the mean loudness between the two genres.

This is a two-tailed test, testing whether one genre is either louder or softer on average than the other.

In [ ]:
# Pairwise T-tests for loudness between genres
from scipy.stats import ttest_ind

results = []

for i, genre_a in enumerate(selected_genres):
    for genre_b in selected_genres[i + 1:]:
        group_a = df[df["genre"] == genre_a]["loudness"]
        group_b = df[df["genre"] == genre_b]["loudness"]
        
        t_stat, p_value = ttest_ind(group_a, group_b, equal_var=False, nan_policy="omit")
        
        results.append({
            "Genre A": genre_a,
            "Genre B": genre_b,
            "T-Statistic": round(t_stat, 3),
            "P-Value": round(p_value, 4),
            "Significant": "Yes" if p_value < 0.05 else "No",
            "Louder Genre": genre_a if group_a.mean() > group_b.mean() else genre_b


        })

ttest_df = pd.DataFrame(results)

print("Loudness Genre Pairwise T-Test Results:")
display(ttest_df)

#### Insights

Hip-Hop/Rap vs. Alternative/Indie shows a statistically significant difference in loudness

- P-value: 0.0389 < 0.05 → We reject the null hypothesis
- **Alternative/Indie is louder than Hip-Hop/Rap**

#### 4.3.4. Effect Size (Cohen’s d)

In [ ]:
effect_sizes = []

for i, genre_a in enumerate(selected_genres):
    for genre_b in selected_genres[i + 1:]:
        group_a = df[df["genre"] == genre_a]["loudness"].dropna()
        group_b = df[df["genre"] == genre_b]["loudness"].dropna()
        
        d = cohens_d(group_a, group_b)
        interpretation = interpret_effect_size(d)

        effect_sizes.append({
            "Genre A": genre_a,
            "Genre B": genre_b,
            "Cohen's d": d,
            "Interpretation": interpretation

        })

effect_sizes_loudness_df = pd.DataFrame(effect_sizes)
display(effect_sizes_loudness_df)

A large effect size supports the conclusion that Alternative/Indie is louder than Hip-Hop/Rap.

**Note: Observed a Large difference between Hip-Hop/Rap and Dance/Electronic**

- A large Cohen's d (–0.930) tells us that the difference in loudness is quite substantial.  

- But T-test p-value (even lowered to 0.01) was not significant, which means:  

    - The difference might be practically meaningful (big gap in values) but there's still too much uncertainty to confidently say it's real with this test.


**Consider:** increasing the sample size—more tracks could reduce noise and help reach significance.


### 4.3 Acousticness between genres

#### 4.3.1. Acousticness Means Comparison

In [ ]:
# Average acousticness scores for selected genres
genre_acousticness = (
    df.groupby("genre")["acousticness"]
    .mean()
    .loc[selected_genres]
    .sort_values(ascending=False)
)

print("Acousticness scores comparison between selected genres:")
display(genre_acousticness.to_frame())

#### 4.3.2. Acousticness Score Distribution

In [ ]:
# Violin plot of acousticness distributions by genre
filtered_df = df[df["genre"].isin(selected_genres)]
genre_medians = filtered_df.groupby("genre")["acousticness"].median()

plt.figure(figsize=(8, 5))
sns.violinplot(data=filtered_df, x="genre", y="acousticness", inner="box")

# Annotate medians
for i, genre in enumerate(selected_genres):
    median = genre_medians[genre]
    plt.text(i, median + 0.1, f"Median: {median:.2f}", ha="center", color="red")

# Plot
plt.title("Acousticness Score Distribution by Genre", fontsize=14)
plt.ylabel("Acousticness")
plt.xlabel("Genre")
plt.xticks(rotation=15)
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

#### Insights
- Pop exhibits the highest median acousticness (0.65)
- Pop and Alternative/Indie show bigger variability in acousticness.
- Dance/Electronic has the tightest distribution, reinforcing its low acousticness across tracks.

#### 4.3.3. Acousticness T-Test Comparison

General Hypotheses for Acousticness Comparison Between Two Genres:

- Null Hypothesis (H₀): There is no difference in the mean acousticness between selected genre pairs.
- Alternative Hypothesis (H₁): There is a difference in the mean acousticness between the two genres.

This is a two-tailed test, testing whether one genre is either more or less acoustic on average than the other.

In [ ]:
# Pairwise T-tests for acousticness between genres
from scipy.stats import ttest_ind

results = []

for i, genre_a in enumerate(selected_genres):
    for genre_b in selected_genres[i + 1:]:
        group_a = df[df["genre"] == genre_a]["acousticness"]
        group_b = df[df["genre"] == genre_b]["acousticness"]
        
        t_stat, p_value = ttest_ind(group_a, group_b, equal_var=False, nan_policy="omit")
        
        results.append({
            "Genre A": genre_a,
            "Genre B": genre_b,
            "T-Statistic": round(t_stat, 3),
            "P-Value": round(p_value, 4),
            "Significant": "Yes" if p_value < 0.05 else "No"
        })

ttest_df = pd.DataFrame(results)

print("Acousticness Genre Pairwise T-Test Results:")
display(ttest_df)

There are three pairs that show a statistically significant difference in acousticness.

- **Alternative/Indie is more acoustic than Dance/Electronic and Hip-Hop/Rap**
- **Pop is more acoustic than Dance/Electronic**

#### 4.3.4. Effect Size (Cohen’s d) for Two Independent Samples

In [ ]:
effect_sizes = []

for i, genre_a in enumerate(selected_genres):
    for genre_b in selected_genres[i + 1:]:
        group_a = df[df["genre"] == genre_a]["acousticness"].dropna()
        group_b = df[df["genre"] == genre_b]["acousticness"].dropna()
        
        d = cohens_d(group_a, group_b)
        interpretation = interpret_effect_size(d)

        effect_sizes.append({
            "Genre A": genre_a,
            "Genre B": genre_b,
            "Cohen's d": d,
            "Interpretation": interpretation

        })

effect_sizes_df = pd.DataFrame(effect_sizes)
display(effect_sizes_df)

#### Overall Insights:
- It is confirmed that **Alternative/Indie** is more acoustic than Dance/Electronic and Hip-Hop/Rap
- Also, Pop is more acoustic than Dance/Electronic
- The **strongest contrast** is between **Alternative/Indie** and **Dance/Electronic**.

Note: Observed a Large difference between Pop and Alternative/Indie, but there is uncertainty to confidently say it's real difference.

## Exploratory Data Analysis Highlights

- **Top Artists:** Billie Eilish, Dua Lipa, and Travis Scott are the most featured artists, each with three songs in the top 50. 
- **Top Album:** Dua Lipa's album, "Future Nostalgia," is the most prominent album with three tracks.
- **Genre Dominance:** Pop and Hip-Hop/Rap are the most dominant genres, making up nearly half of the top tracks.
- The dataset contains 16 unique genres, with 10 of them appearing only once.

Key Audio Features:

- Danceability: **A majority of the top tracks (64%) are highly danceable.** Hip-Hop/Rap tracks are significantly more danceable than Pop tracks.
- Loudness: Top tracks are generally loud. Alternative/Indie tracks are, on average, louder than Hip-Hop/Rap tracks.
- Acousticness: **Alternative/Indie tracks have the highest acousticness**, while Dance/Electronic tracks have the lowest.

Feature Correlations:

- There is a strong positive correlation between a track's energy and its loudness.
- There is a strong negative correlation between energy and acousticness, meaning more acoustic songs tend to be less energetic.

The song lengths in the top 50 vary, with:
- Longest: "Sicko Mode" by Travis Scott (5:12)
- Shortest: "Mood" by 24kGoldn (2:20).

### Recommendations

For Marketing & Promotional Strategies:

- **Leverage Top Performers:** Feature the top artists (Billie Eilish, Dua Lipa, Travis Scott) and the top album ("Future Nostalgia") in marketing campaigns to capitalize on their popularity.
- **Create Themed Campaigns:** Develop marketing initiatives based on audio features like "high-danceability" playlists or "high-energy" workout mixes to target specific listener preferences.
- **Target Genre-Specific Audiences:** Use the genre insights to tailor marketing efforts. For example, promote high-danceability Hip-Hop/Rap tracks to audiences interested in dance and parties.

For Data Analysis & Future Research:

- **Explore Sub-Genres:** Investigate the dominant "Pop" category further to identify which sub-genres are most successful.
- **Analyze Audio Feature Drivers:** Conduct a deeper analysis to understand the specific musical elements (like BPM, rhythm, and instrumentation) that contribute to high or low scores in danceability and acousticness.
- **Expand the Dataset:** To validate these trends, analyze a larger and more diverse dataset of popular music from different years or regions.